<center>
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Slack Community</a>
    </p>
</center>

# **Arize Agent Mastry Course: Trace your First Agent**

In this first lab, we will create a simple travel agent powered by the Agno framework and OpenAI models. We’ll begin by installing the necessary OpenInference packages and setting up tracing with Arize.

Next, we’ll define a set of basic tools that provide destination information, estimate trip budgets, and suggest local activities.

Finally, we’ll build and run our agent, viewing the resulting trace outputs in Arize to understand how the agent uses its tools and reasoning.

You will need a free Arize account, an OpenAI API key, and a free [Tavily](https://auth.tavily.com/) API Key.

# Set up keys and dependenies

In [ ]:
!pip install -qq arize-otel agno openai openinference-instrumentation-agno openinference-instrumentation-openai httpx

In [ ]:
import os
from getpass import getpass
from google.colab import userdata

os.environ["ARIZE_SPACE_ID"] = userdata.get("ARIZE_SPACE_ID") or getpass("🔑 Enter your Arize Space ID: ")

os.environ["ARIZE_API_KEY"] = userdata.get("ARIZE_API_KEY") or getpass("🔑 Enter your Arize API Key: ")

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY") or getpass("🔑 Enter your OpenAI API Key: ")

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY") or getpass("🔑 Enter your Tavily API Key: ")

# Setup tracing

In [ ]:
from arize.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.instrumentation.agno import AgnoInstrumentor

model_id = "travel-agent-demo"
tracer_provider = register(
    space_id=os.getenv("ARIZE_SPACE_ID"),
    api_key=os.getenv("ARIZE_API_KEY"),
    project_name=model_id,
    #set_global_tracer_provider=True,
    log_to_console=True,
    endpoint="https://otlp.ca-central-1a.arize.com/v1"
)
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)
AgnoInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: travel-agent-demo
|  Span Processor: Multiple Span Processors
|  Collector Endpoint: Multiple Span Exporters
|  Transport: Multiple Span Exporters
|  Transport Headers: Multiple Span Exporters
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [ ]:
from opentelemetry import trace
tracer = trace.get_tracer(__name__)

# Define tools

First, we’ll define a few helper functions to support our tools. In particular, we’ll use Tavily Search to help the tools gather general information about each destination.

In [ ]:
# --- Helper functions for tools ---
import httpx

def _search_api(query: str) -> str | None:
    """Try Tavily search first, fall back to None."""
    tavily_key = os.getenv("TAVILY_API_KEY")
    if not tavily_key:
        return None
    try:
        resp = httpx.post(
            "https://api.tavily.com/search",
            json={
                "api_key": tavily_key,
                "query": query,
                "max_results": 3,
                "search_depth": "basic",
                "include_answer": True,
            },
            timeout=8,
        )
        data = resp.json()
        answer = data.get("answer") or ""
        snippets = [r.get("content", "") for r in data.get("results", [])]
        combined = " ".join([answer] + snippets).strip()
        return combined[:400] if combined else None
    except Exception:
        return None


def _compact(text: str, limit: int = 200) -> str:
    """Compact text for cleaner outputs."""
    cleaned = " ".join(text.split())
    return cleaned if len(cleaned) <= limit else cleaned[:limit].rsplit(" ", 1)[0]


Our agent will have access to three tools, which we’ll continue to enhance in upcoming labs:

1. Essential Info – Provides key travel details about the destination, such as weather and general conditions.

2. Budget Basics – Offers insights into travel costs and helps plan budgets based on selected activities.

3. Local Flavor – Recommends unique local experiences and cultural highlights.

In [ ]:
from agno.tools import tool

@tool
def essential_info(destination: str) -> str:
    """Get basic travel info (weather, best time, attractions, etiquette)."""
    q = f"{destination} travel essentials weather best time top attractions etiquette"
    s = _search_api(q)
    if s:
        return f"{destination} essentials: {_compact(s)}"

    return f"{destination} is a popular travel destination. Expect local culture, cuisine, and landmarks worth exploring."

@tool
def budget_basics(destination: str, duration: str) -> str:
    """Summarize travel cost categories."""
    q = f"{destination} travel budget average daily costs {duration}"
    s = _search_api(q)
    if s:
        return f"{destination} budget ({duration}): {_compact(s)}"
    return f"Budget for {duration} in {destination} depends on lodging, meals, transport, and attractions."

@tool
def local_flavor(destination: str, interests: str = "local culture") -> str:
    """Suggest authentic local experiences."""
    q = f"{destination} authentic local experiences {interests}"
    s = _search_api(q)
    if s:
        return f"{destination} {interests}: {_compact(s)}"
    return f"Explore {destination}'s unique {interests} through markets, neighborhoods, and local eateries."


# Define agent

Next, we’ll construct our agent. The Agno framework makes this process straightforward by allowing us to easily define key parameters such as the model, instructions, and tools.

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat

# --- Main Agent ---
trip_agent = Agent(
    name="TripPlanner",
    role="AI Travel Assistant",
    model=OpenAIChat(id="gpt-4.1"),
    instructions=(
        "You are a friendly and knowledgeable travel planner. "
        "Combine multiple tools to create a trip plan including essentials, budget, and local flavor. "
        "Keep the tone natural, clear, and under 1000 words."
    ),
    markdown=True,
    tools=[essential_info, budget_basics, local_flavor],
)



# Run agent

Finally, we are ready to run our agent! Run this cell to see an example in action.

In [ ]:
# --- Example usage ---
destination = "Tokyo"
duration = "5 days"
interests = "food, culture"

query = f"""
Plan a {duration} trip to {destination}.
Focus on {interests}.
Include essential info, budget breakdown, and local experiences.
"""
trip_agent.print_response(
    query,
    stream = True
)

Output()

{
    "name": "ChatCompletion",
    "context": {
        "trace_id": "0xb80bf4eb1b6225cc619ae72f7ec69eed",
        "span_id": "0x85bffe3959bd8979",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0xd8e4d134fdc6c71e",
    "start_time": "2026-01-02T14:10:51.850690Z",
    "end_time": "2026-01-02T14:10:54.032601Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "llm.provider": "openai",
        "llm.system": "openai",
        "input.value": "{\"messages\": [{\"role\": \"developer\", \"content\": \"<your_role>\\nAI Travel Assistant\\n</your_role>\\n\\n<instructions>\\nYou are a friendly and knowledgeable travel planner. Combine multiple tools to create a trip plan including essentials, budget, and local flavor. Keep the tone natural, clear, and under 1000 words.\\n</instructions>\\n\\n<additional_information>\\n- Use markdown to format your answers.\\n</additional_information>\"}, {\"role\": \"user\", \"content\": \"\\nPl

# Observe

Within the Arize platform, you can explore every step your agent takes from LLM calls and tool executions to inputs, outputs, and intermediate reasoning steps.

This detailed trace view allows you to understand how the agent arrived at its final answer, providing transparency into its decision-making process. You can also monitor key metrics such as cost, latency, and metadata to evaluate performance and optimize future runs.

![Agent Trace](https://storage.googleapis.com/arize-phoenix-assets/assets/images/course-agno-agent-1.png)